In [26]:
import random
import pandas as pd
import numpy as np
import time

In [27]:
def diff(Rule1, Rule2):
    if Rule1[-1] != Rule2[-1]:
        return -1
    m = len(Rule1)
    count = 0
    for i in range(m-1):
        if Rule1[i] == Rule2[i]:
            count += 1
    return count/m

In [38]:
def sampling(base, ran, e, k = 2):
    

    num = len(base)
    R = []
    list_index = []
    while (len(R)<num*ran/100):
        index = random.randrange(num)
        while (index in list_index):
            index = random.randrange(num)
        # list_index.append(index)
        T = []
        T.append(index)
        for i in range(index-k,index+k+1):
            if i < num:
                if i in list_index:
                    continue
                else:
                    if diff(base[i],base[index]) < 1 - e:
                        T.append(i)
        for i in T:
            temp = 0
            for j in range(len(R)):
                if diff(base[i],R[j]) < 1 - e:
                    continue
                else:
                    temp = 1
            if temp:
                T.remove(i)
        for i in T:
            R.append(base[i])
            list_index.append(i)
    
    return R



In [29]:
def combination(k, n):
    if k == 0 or k == n:
        return 1
    if k == 1:
        return n
    return combination(k - 1, n - 1) + combination(k, n - 1)


def caculateA(base):
    colum = len(base[0])
    row = len(base)
    A = np.zeros((row, combination(4, colum - 1)))

    for r1 in range(row):
        k = [0] * combination(4, colum - 1)
        temp = 0
        for a in range(0, colum - 4):
            for b in range(a + 1, colum - 3):
                for c in range(b + 1, colum - 2):
                    for d in range(c + 1, colum - 1):
                        for r2 in range(row):
                            if base[r1][a] == base[r2][a] and base[r1][b] == base[r2][b] and base[r1][c] == base[r2][c] and base[r1][d] == base[r2][d]:
                                k[temp] += 1

                        A[r1][temp] = k[temp] / row
                        temp += 1
    print("done A")
    return A


def caculateM(base):
    colum = len(base[0])
    row = len(base)
    M = np.zeros((row, colum - 1))
    for t1 in range(row):
        k = [0] * (colum - 1)
        temp = 0
        for i in range(colum - 1):
            for t2 in range(row):
                if base[t1][i] == base[t2][i] and base[t1][colum - 1] == base[t2][colum - 1]:
                    k[temp] += 1
            M[t1][temp] = k[temp] / row
            temp += 1

    return M


def caculateB(base, A, M):
    colum = len(base[0])
    row = len(base)
    B = np.zeros((row, combination(3, colum - 1)))

    for r in range(row):
        temp = 0
        for a in range(0, colum - 3):
            for b in range(a + 1, colum - 2):
                for c in range(b + 1, colum - 1):
                    B[r][temp] = sum(A[r]) * min(M[r][a], M[r][b], M[r][c])
                    temp += 1
    print("done B")
    return B


def caculateC(base, B):
    colum = len(base[0])
    row = len(base)
    cols = 2 * combination(3, colum - 1)
    C = np.zeros((row, cols))

    for r1 in range(row):
        temp = 0
        for i in range(2):
            for a in range(0, (colum - 3)):
                for b in range(a + 1, (colum - 2)):
                    for c in range(b + 1, (colum - 1)):
                        for r2 in range(row):
                            if base[r1][a] == base[r2][a] and base[r1][b] == base[r2][b] and base[r1][c] == base[r2][c] and base[r2][colum - 1] == i:
                                C[r1][temp] += B[r2][temp % combination(3, colum - 1)]
                        # print(temp,":",temp//combination(3,colum-1))
                        temp += 1
    print("done C")
    return C

In [30]:
def FISA(base, C, list):
    colum = len(base[0])
    row = len(base)

    cols = combination(3, (colum - 1))
    C0 = [0] * cols
    C1 = [0] * cols

    t = 0
    for a in range(0, colum - 3):
        for b in range(a + 1, colum - 2):
            for c in range(b + 1, colum - 1):
                for r in range(row-1):
                    if base[r][a] == list[a] and base[r][b] == list[b] and base[r][c] == list[c] and base[r][colum-1] == 0:
                        C0[t] = C[r][t + 0 * cols]
                        # break
                    if base[r][a] == list[a] and base[r][b] == list[b] and base[r][c] == list[c] and base[r][colum-1] == 1:
                        C1[t] = C[r][t + 1 * cols]
                        # break
                t += 1
    #print(t)

    D0 = max(C0) + min(C0)
    D1 = max(C1) + min(C1)

    #print(D0, max(C0), min(C0))
    #print(D1, max(C1), min(C1))
    #print(D2, max(C2), min(C2))
    if D0 > 3*D1:
        return 0, D0/(D0+D1)
    else:
        return 1, D1/(D0+D1)

In [31]:
def Acc(A,B):
    result = 0

    for i in range(len(A)):
        if int(A[i]) - int(B[i]) == 0:
            result += 1

    return round(result*100/len(A), 2)


def Tprecision(Pre,Act):
    result = 0
    TP = 0
    FP = 0

    for i in range(len(Pre)):
        if int(Pre[i]) == 1 and int(Act[i]) == 1:
            TP +=1
        if int(Pre[i]) == 1 and int(Act[i]) == 0:
            FP +=1

    #return str(TP) +  " : " + str(TP+FP)

    if TP:
        return round(100*TP/(TP+FP),2)
    else:
        if FP:
            return 0
        else:
            return None


def Trecall(Pre,Act):
    result = 0
    TP = 0
    FN = 0

    for i in range(len(Pre)):
        if int(Pre[i]) == 1 and int(Act[i]) == 1:
            TP +=1
        if int(Pre[i]) == 0 and int(Act[i]) == 1:
            FN +=1

    #return str(TP) +  " : " + str(TP+FN)

    if TP:
        return round(100*TP/(TP+FN),2)
    else:
        if FN:
            return 0
        else:
            return None

# listAcc = []
# listPre = []
# listRe = []
# timeTest = []
# timeUpdate = []
# listRank = []
# res = []

def testAccuracy(base,Te,C):
    test = Te
    X = np.zeros(len(test))
    ddd = np.zeros(len(test))
    X_test = np.array(test).T[-1]
    for i in range(len(test)):
        X[i], ddd[i] = FISA(base, C, test[i])
        # listRank.append(ddd[i])
        #print(test[i])
    # for dd in ddd:
    #     print(dd)
    # listRank.append(ddd)

    #print(X)
    #print(X_test)
    # res.append(X)
    # listAcc.append(Acc(X,X_test))
    # listPre.append(Tprecision(X,X_test))
    # listRe.append(Trecall(X,X_test))

    return Acc(X,X_test)

In [32]:
def scenario_FKGS(ran, e):

    traindf = pd.read_csv('train.csv')
    testdf = pd.read_csv('test.csv')
    train = traindf.values.tolist()
    test = testdf.values.tolist()


    sampling_time = []
    train_time = []
    test_time = []
    accuracy = []
    for i in range(10):
        start = time.time()
        base = sampling(ran=ran, base=train, e=e)
        totalTime = time.time() - start
        sampling_time.append(totalTime)
        print(f'len base {i}:', len(base))


        start = time.time()
        A = caculateA(base)
        M = caculateM(base)
        B = caculateB(base,A,M)
        C = caculateC(base,B)
        totalTime = time.time() - start
        train_time.append(totalTime)


        start = time.time()
        acc = testAccuracy(base,test,C)
        totalTime = time.time() - start
        accuracy.append(acc)
        test_time.append(totalTime)


    print('sampling_time:', sampling_time)
    print('train_time:', train_time)
    print('test_time:', test_time)
    print('accuracy:', accuracy)


In [33]:
def scenario_random(ran):

    traindf = pd.read_csv('train.csv')
    testdf = pd.read_csv('test.csv')
    test = testdf.values.tolist()

    sampling_time = []
    train_time = []
    test_time = []
    accuracy = []
    for i in range(10):
        
        start = time.time()
        train = traindf.sample(n=int(len(traindf)*ran/100))
        base = train.values.tolist()
        totalTime = time.time() - start
        sampling_time.append(totalTime)
        print(f'len base {i}:', len(base))


        start = time.time()
        A = caculateA(base)
        M = caculateM(base)
        B = caculateB(base,A,M)
        C = caculateC(base,B)
        totalTime = time.time() - start
        train_time.append(totalTime)


        start = time.time()
        acc = testAccuracy(base,test,C)
        totalTime = time.time() - start
        accuracy.append(acc)
        test_time.append(totalTime)

    print('sampling_time:', sampling_time)
    print('train_time:', train_time)
    print('test_time:', test_time)
    print('accuracy:', accuracy)


In [34]:
scenario_FKGS(10, 0.1)

len base 0: 1106
done A
done B
done C
len base 1: 1109
done A
done B
done C
len base 2: 1110
done A
done B
done C
len base 3: 1109
done A
done B
done C
len base 4: 1108
done A
done B
done C
len base 5: 1107
done A
done B
done C
len base 6: 1105
done A
done B
done C
len base 7: 1106
done A
done B
done C
len base 8: 1107
done A
done B
done C
len base 9: 1110
done A
done B
done C
sampling_time: [0.25500035285949707, 0.23176264762878418, 0.22499775886535645, 0.23804759979248047, 0.2335221767425537, 0.22699928283691406, 0.2310028076171875, 0.22900009155273438, 0.23200035095214844, 0.23199963569641113]
train_time: [31.069599390029907, 31.750704050064087, 31.479002952575684, 31.412015199661255, 31.01317858695984, 30.41343927383423, 31.42226266860962, 30.517969369888306, 32.17427062988281, 32.30485558509827]
test_time: [32.68796443939209, 32.84789037704468, 32.7702956199646, 32.65174317359924, 32.53800582885742, 32.55918312072754, 32.639607667922974, 32.37109327316284, 32.76636981964111, 32.68

In [35]:
scenario_FKGS(10, 0.2)

len base 0: 1106
done A
done B
done C
len base 1: 1105
done A
done B
done C
len base 2: 1106
done A
done B
done C
len base 3: 1107
done A
done B
done C
len base 4: 1107
done A
done B
done C
len base 5: 1108
done A
done B
done C
len base 6: 1107
done A
done B
done C
len base 7: 1106
done A
done B
done C
len base 8: 1107
done A
done B
done C
len base 9: 1105
done A
done B
done C
sampling_time: [0.24699735641479492, 0.24912810325622559, 0.2529938220977783, 0.25300073623657227, 0.24199318885803223, 0.24299979209899902, 0.24300003051757812, 0.2497270107269287, 0.2440016269683838, 0.23999905586242676]
train_time: [26.45424199104309, 25.821566581726074, 26.020081996917725, 26.772688150405884, 26.751339197158813, 26.55585265159607, 25.94494390487671, 25.91187024116516, 26.283520460128784, 26.21224045753479]
test_time: [31.27181363105774, 31.45109534263611, 31.151880741119385, 31.57888388633728, 31.42690682411194, 31.33870267868042, 31.099430799484253, 31.13206648826599, 31.23501682281494, 31.6

In [39]:
scenario_FKGS(10, 0.3)

len base 0: 1106
done A
done B
done C
len base 1: 1106
done A
done B
done C
len base 2: 1106
done A
done B
done C
len base 3: 1106
done A
done B
done C
len base 4: 1105
done A
done B
done C
len base 5: 1107
done A
done B
done C
len base 6: 1106
done A
done B
done C
len base 7: 1107
done A
done B
done C
len base 8: 1106
done A
done B
done C
len base 9: 1105
done A
done B
done C
sampling_time: [0.2977588176727295, 0.27900052070617676, 0.2859487533569336, 0.2789306640625, 0.2820000648498535, 0.28299975395202637, 0.2861177921295166, 0.267042875289917, 0.28000473976135254, 0.27899980545043945]
train_time: [27.4707670211792, 26.612672567367554, 26.276891708374023, 26.515198707580566, 26.80617117881775, 26.327840328216553, 26.028281450271606, 25.84854245185852, 26.00935983657837, 25.587698698043823]
test_time: [31.531574726104736, 31.55915069580078, 31.459521770477295, 31.548287630081177, 31.621367931365967, 32.90231490135193, 31.37862491607666, 31.85826301574707, 31.45894765853882, 31.323895

In [40]:
scenario_FKGS(15, 0.1)

len base 0: 1660
done A
done B
done C
len base 1: 1661
done A
done B
done C
len base 2: 1659
done A
done B
done C
len base 3: 1661
done A
done B
done C
len base 4: 1662
done A
done B
done C
len base 5: 1661
done A
done B
done C
len base 6: 1659
done A
done B
done C
len base 7: 1661
done A
done B
done C
len base 8: 1657
done A
done B
done C
len base 9: 1661
done A
done B
done C
sampling_time: [0.5149979591369629, 0.5186553001403809, 0.5299718379974365, 0.525334358215332, 0.5179922580718994, 0.51641845703125, 0.5230073928833008, 0.5114071369171143, 0.5191493034362793, 0.5124046802520752]
train_time: [68.9643497467041, 72.90253591537476, 70.08287692070007, 69.69951510429382, 69.8340311050415, 68.27145767211914, 71.53910803794861, 67.97696924209595, 68.4840133190155, 69.61378026008606]
test_time: [48.689340114593506, 49.28900957107544, 48.80730843544006, 48.95177435874939, 49.04672336578369, 48.8215217590332, 49.1927535533905, 48.596245765686035, 48.53386926651001, 48.73775100708008]
accur

In [41]:
scenario_FKGS(15, 0.2)

len base 0: 1660
done A
done B
done C
len base 1: 1659
done A
done B
done C
len base 2: 1658
done A
done B
done C
len base 3: 1658
done A
done B
done C
len base 4: 1659
done A
done B
done C
len base 5: 1658
done A
done B
done C
len base 6: 1660
done A
done B
done C
len base 7: 1660
done A
done B
done C
len base 8: 1657
done A
done B
done C
len base 9: 1657
done A
done B
done C
sampling_time: [0.5734267234802246, 0.5693991184234619, 0.5767793655395508, 0.5486428737640381, 0.5500082969665527, 0.5632739067077637, 0.5610699653625488, 0.5639998912811279, 0.5618143081665039, 0.5624632835388184]
train_time: [57.8249671459198, 57.648451805114746, 58.268147230148315, 57.879201889038086, 57.98568272590637, 58.90938186645508, 58.07468843460083, 57.98478865623474, 57.77824401855469, 58.21957850456238]
test_time: [46.99012541770935, 46.92669367790222, 46.917083978652954, 46.96510434150696, 46.941895961761475, 47.20848727226257, 46.909979820251465, 47.011335611343384, 47.02325797080994, 46.977219343

In [42]:
scenario_FKGS(15, 0.3)

len base 0: 1657
done A
done B
done C
len base 1: 1657
done A
done B
done C
len base 2: 1658
done A
done B
done C
len base 3: 1657
done A
done B
done C
len base 4: 1658
done A
done B
done C
len base 5: 1658
done A
done B
done C
len base 6: 1657
done A
done B
done C
len base 7: 1658
done A
done B
done C
len base 8: 1658
done A
done B
done C
len base 9: 1658
done A
done B
done C
sampling_time: [0.6560423374176025, 0.6460645198822021, 0.6454086303710938, 0.6380000114440918, 0.6492199897766113, 0.6507959365844727, 0.6500072479248047, 0.652047872543335, 0.6599223613739014, 0.6463451385498047]
train_time: [59.71023654937744, 58.565502643585205, 59.601731300354004, 58.767008781433105, 60.334192752838135, 59.10899114608765, 59.27102708816528, 59.51765036582947, 59.47526216506958, 58.4378342628479]
test_time: [47.38912010192871, 47.15640139579773, 47.40141725540161, 47.16002583503723, 47.6774377822876, 47.31197118759155, 47.43720030784607, 49.27919030189514, 47.72749400138855, 46.36348628997803

In [43]:
scenario_FKGS(20, 0.1)

len base 0: 2214
done A
done B
done C
len base 1: 2209
done A
done B
done C
len base 2: 2210
done A
done B
done C
len base 3: 2213
done A
done B
done C
len base 4: 2210
done A
done B
done C
len base 5: 2214
done A
done B
done C
len base 6: 2209
done A
done B
done C
len base 7: 2209
done A
done B
done C
len base 8: 2209
done A
done B
done C
len base 9: 2209
done A
done B
done C
sampling_time: [0.8914566040039062, 0.8888635635375977, 0.9013381004333496, 0.8950145244598389, 0.9004662036895752, 0.8829259872436523, 0.9101176261901855, 0.8760063648223877, 0.8985433578491211, 0.914161205291748]
train_time: [121.60007500648499, 118.81085085868835, 122.72993087768555, 122.36982345581055, 120.15006947517395, 118.9786274433136, 120.67894005775452, 119.54916572570801, 125.25247478485107, 122.98049902915955]
test_time: [63.915486574172974, 63.53458881378174, 65.3000123500824, 64.0712161064148, 63.38939571380615, 63.43702220916748, 63.487088441848755, 63.527911901474, 66.62753367424011, 66.269877672

In [44]:
scenario_FKGS(20, 0.2)

len base 0: 2210
done A
done B
done C
len base 1: 2211
done A
done B
done C
len base 2: 2211
done A
done B
done C
len base 3: 2210
done A
done B
done C
len base 4: 2210
done A
done B
done C
len base 5: 2210
done A
done B
done C
len base 6: 2209
done A
done B
done C
len base 7: 2209
done A
done B
done C
len base 8: 2210
done A
done B
done C
len base 9: 2211
done A
done B
done C
sampling_time: [1.0658612251281738, 1.0147826671600342, 1.0059828758239746, 1.0187981128692627, 1.0113005638122559, 1.1301319599151611, 1.019946575164795, 1.0044286251068115, 1.002244472503662, 1.0003018379211426]
train_time: [100.76340842247009, 103.26364016532898, 100.69800734519958, 103.3094711303711, 103.49316382408142, 104.22089385986328, 103.34450793266296, 103.09596753120422, 100.63470721244812, 102.37105965614319]
test_time: [63.77519464492798, 63.78176999092102, 63.06373119354248, 63.15795540809631, 65.67403817176819, 64.23923015594482, 63.13786435127258, 63.34390115737915, 62.842597007751465, 63.3622193

In [45]:
scenario_FKGS(20, 0.3)

len base 0: 2209
done A
done B
done C
len base 1: 2210
done A
done B
done C
len base 2: 2210
done A
done B
done C
len base 3: 2210
done A
done B
done C
len base 4: 2209
done A
done B
done C
len base 5: 2210
done A
done B
done C
len base 6: 2209
done A
done B
done C
len base 7: 2210
done A
done B
done C
len base 8: 2210
done A
done B
done C
len base 9: 2209
done A
done B
done C
sampling_time: [1.178117275238037, 1.2212066650390625, 1.1847732067108154, 1.1831581592559814, 1.1990132331848145, 1.2040271759033203, 1.161195993423462, 1.177807092666626, 1.1929774284362793, 1.2038395404815674]
train_time: [107.34545803070068, 108.36347270011902, 105.64465570449829, 108.02228260040283, 107.98591494560242, 108.49587392807007, 108.32672691345215, 110.44421362876892, 112.34078311920166, 111.08011627197266]
test_time: [64.38037705421448, 64.11132311820984, 64.10220861434937, 63.960153341293335, 64.1078748703003, 64.21646666526794, 64.28493571281433, 66.82856035232544, 66.49352359771729, 66.17875003

In [46]:
scenario_random(10)

len base 0: 1104
done A
done B
done C
len base 1: 1104
done A
done B
done C
len base 2: 1104
done A
done B
done C
len base 3: 1104
done A
done B
done C
len base 4: 1104
done A
done B
done C
len base 5: 1104
done A
done B
done C
len base 6: 1104
done A
done B
done C
len base 7: 1104
done A
done B
done C
len base 8: 1104
done A
done B
done C
len base 9: 1104
done A
done B
done C
sampling_time: [0.0009999275207519531, 0.0009999275207519531, 0.0, 0.001001119613647461, 0.0009944438934326172, 0.0010020732879638672, 0.0010004043579101562, 0.0009295940399169922, 0.0009999275207519531, 0.0009999275207519531]
train_time: [30.644283056259155, 31.551743745803833, 32.30472993850708, 33.2475266456604, 33.29711985588074, 32.69009208679199, 32.012417793273926, 31.09705090522766, 31.899665117263794, 31.68399930000305]
test_time: [33.51324439048767, 33.45420980453491, 33.80693817138672, 33.95367360115051, 33.933791637420654, 33.969523191452026, 33.752164125442505, 33.627259731292725, 33.713810443878174,

In [47]:
scenario_random(15)

len base 0: 1656
done A
done B
done C
len base 1: 1656
done A
done B
done C
len base 2: 1656
done A
done B
done C
len base 3: 1656
done A
done B
done C
len base 4: 1656
done A
done B
done C
len base 5: 1656
done A
done B
done C
len base 6: 1656
done A
done B
done C
len base 7: 1656
done A
done B
done C
len base 8: 1656
done A
done B
done C
len base 9: 1656
done A
done B
done C
sampling_time: [0.0009999275207519531, 0.00099945068359375, 0.0009999275207519531, 0.001001119613647461, 0.0015556812286376953, 0.0010066032409667969, 0.0010035037994384766, 0.0010013580322265625, 0.0009992122650146484, 0.001001119613647461]
train_time: [72.44538307189941, 72.06383228302002, 72.31767725944519, 73.22060036659241, 70.05993175506592, 70.8377935886383, 69.75901317596436, 71.27478981018066, 72.36639595031738, 72.30941414833069]
test_time: [50.76183867454529, 50.92787051200867, 51.30626320838928, 50.4125497341156, 49.98277831077576, 50.49592351913452, 49.03752851486206, 50.90210318565369, 51.0681791305

In [48]:
scenario_random(20)

len base 0: 2208
done A
done B
done C
len base 1: 2208
done A
done B
done C
len base 2: 2208
done A
done B
done C
len base 3: 2208
done A
done B
done C
len base 4: 2208
done A
done B
done C
len base 5: 2208
done A
done B
done C
len base 6: 2208
done A
done B
done C
len base 7: 2208
done A
done B
done C
len base 8: 2208
done A
done B
done C
len base 9: 2208
done A
done B
done C
sampling_time: [0.002002716064453125, 0.0010077953338623047, 0.0019969940185546875, 0.0009999275207519531, 0.0010004043579101562, 0.0010004043579101562, 0.0015714168548583984, 0.0010006427764892578, 0.001999378204345703, 0.0009999275207519531]
train_time: [126.6957950592041, 127.0359160900116, 128.0400264263153, 125.32807087898254, 126.35509705543518, 126.17212128639221, 126.28278970718384, 128.57989645004272, 126.41831612586975, 127.947336435318]
test_time: [68.13509583473206, 67.81697845458984, 67.67295980453491, 67.64972257614136, 67.53515028953552, 67.8500919342041, 67.90429639816284, 67.9761552810669, 67.775